In [242]:
import pandas as pd 
import numpy as np
import re

In [243]:
df = pd.read_excel('BIEU THUE XNK 2016.xlsx', sheet_name="CHUONG 98", header=4).dropna(how='all').fillna("")
df['-1-'] = df['-1-'].astype('str')
df['-2-'] = df['-2-'].astype('str')
df["-2'-"] = df["-2'-"].astype('str')

df['-4-'] = df['-4-'].astype('str')
df['-5-'] = df['-5-'].astype('str')
df["-6-"] = df["-6-"].astype('str')
df["-7-"] = df["-7-"].astype('str')

In [ ]:
df["-1-"] = df['-1-'].str.replace('.', '').str.slice(0,4)
df["-2-"] = df['-2-'].str.replace('.', '').str.slice(0,2)
df["-2'-"] = df["-2'-"].str.replace('.', '').str.slice(0,2)

df['hs'] =  df["-1-"] + df["-2-"] + df["-2'-"]


def related_hs(row):
    if not re.match('^[\d. ]+$', row["-4-"].strip()):
        row['related_hs'] = row["-4-"]
        return row
    row["-4-"] = ("0" + row['-4-']).split(".")[0][-4:]
    row["-5-"] = ("0" + row['-5-']).split(".")[0][-2:]
    row["-6-"] = ("0" + row["-6-"]).split(".")[0][-2:]
    if re.match('^0+$', row["-4-"] + row["-5-"] + row["-6-"]):
        row['related_hs'] = ""
        return row
    row['related_hs'] =  row["-4-"] + row["-5-"] + row["-6-"]
    return row

df = df.apply(related_hs, axis=1)
df = df.drop(['Unnamed: 0', '-1-', '-2-', "-2'-", '-4-', '-5-', '-6-'], axis=1)
df = df.rename(columns={"-3-": "description", "-7-": "tax_rate"})


In [245]:
def insert_row(df, pos, row):
    df2 = pd.DataFrame(np.insert(df.values, pos, values=row, axis=0))
    df2.columns = df.columns
    return df2

In [246]:
def fix_heading(df):
    for index, row in df.iterrows():
        if len(str(row['hs'])) > 5 and not row['description'].startswith('-'):
            new_row = row.copy()
            new_row['hs'] = row['hs'][:2] + '.' + row['hs'][2:4]
            df['description'][index] = "- " + row['description']
            return fix_heading(insert_row(df, index, new_row))
    return df

def fix_hs(row):
    hs = row['hs'].replace('.', '')
    related_hs = row['related_hs'].replace('.', '')

    if len(hs) == 4:
        hs = hs[:2] + '.' + hs[2:4]
    elif len(hs) > 4 and len(hs) < 7:
        hs = hs[:4] + '.' + hs[4:]
    else:
        hs = hs[:4] + '.' + hs[4:6] + '.' + hs[6:]

    if len(related_hs) == 4:
        related_hs = related_hs[:2] + '.' + related_hs[2:4]
    elif len(related_hs) > 4 and len(related_hs) < 7:
        related_hs = related_hs[:4] + '.' + related_hs[4:]
    else:
        related_hs = related_hs[:4] + '.' + related_hs[4:6] + '.' + related_hs[6:]

    if len(row['hs']) < 11 and row['hs'] != "":
        row['hs'] = hs
    if len(row['related_hs']) < 11 and row['related_hs'] != "":
        row['related_hs'] = related_hs

    return row

df = fix_heading(df)
df = df.apply(fix_hs, axis=1)


# df = df.fillna(method='ffill')


cols = ['hs', 'description', 'related_hs', 'tax_rate']
df[cols].to_excel('chapter98.xlsx', index=False)